In [1]:
import gym
import random
import sys
import numpy as np
import tensorflow as tf
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)
tmp=None
import matplotlib.pyplot as plt

In [2]:
def getStuff(samples):
    k=np.array(samples)
    g=np.array(k[:,1],dtype=np.int32)
    dis=[[1,0],[0,1]]
    return np.array([np.array(i) for i in k[:,0]]),np.array([dis[i] for i in g]),np.array(k[:,2]),np.array([np.array(i) for i in k[:,3]]),np.array(k[:,4])

In [3]:
def getVar(sess,name,scope):
    if not scope:
        return sess.run(tf.get_default_graph().get_tensor_by_name(name+':0'))
    else:
        return tf.get_default_graph().get_tensor_by_name(name+':0')

In [4]:
class DQN:
    
    REPLAY_MEMORY_SIZE = 1000             # number of tuples in experience replay  
    EPSILON = 0.8                         # epsilon of epsilon-greedy exploation
    EPSILON_DECAY = 0.99                 # exponential decay multiplier for epsilon
    HIDDEN1_SIZE = 128                     # size of hidden layer 1
    HIDDEN2_SIZE = 128                     # size of hidden layer 2
    EPISODES_NUM = 300                 # number of episodes to train on. Ideally shouldn't take longer than 2000
    MAX_STEPS = 200                     # maximum number of steps in an episode 
    LEARNING_RATE = 0.0001                 # learning rate and other parameters for SGD/RMSProp/Adam
    MINIBATCH_SIZE = 100                 # size of minibatch sampled from the experience replay
    DISCOUNT_FACTOR = 0.9                 # MDP's gamma
    TARGET_UPDATE_FREQ = 100             # number of steps (not episodes) after which to update the target networks 
    LOG_DIR = './logs'       
    # Create and initialize the environment
    def __init__(self, env):
        self.env = gym.make(env)
        assert len(self.env.observation_space.shape) == 1
        self.input_size = self.env.observation_space.shape[0]        # In case of cartpole, 4 state features
        self.output_size = self.env.action_space.n                    # In case of cartpole, 2 actions (right/left)
    
    # Create the Q-network
    def initialize_network(self):
          # placeholder for the state-space input to the q-network
        self.x = tf.placeholder(tf.float32, [None, self.input_size])
        self.action_mask= tf.placeholder(tf.float32,[None,self.output_size])
        self.y = tf.placeholder(tf.float32, [None,])
        #running net
        
        self.Layer1 = tf.layers.dense(self.x,self.HIDDEN1_SIZE,activation=tf.nn.tanh,name='layer1')
        self.Layer2 = tf.layers.dense(self.Layer1,self.HIDDEN2_SIZE,activation=tf.nn.tanh,name='layer2')
        self.Q = tf.layers.dense(self.Layer2,self.output_size,name='output',activation=None)
        
        
        #self.y = self.r+ self.DISCOUNT_FACTOR * tf.reduce_max(self.Q_target,axis=1)

        self.y_dash = tf.reduce_sum(self.Q*self.action_mask,axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.y-self.y_dash))
        
        optimizer = tf.train.AdamOptimizer(self.LEARNING_RATE)

        global_step = tf.Variable(0, name='global_step', trainable=False)
        self.trainer = optimizer.minimize(self.loss, global_step=global_step)
    def targetNetwork(self,t):
        for i in range(len(self.w)):
            if i+1==len(self.w):
                t=np.matmul(t,self.w[i])+self.b[i]
            else:
                t=np.tanh(np.matmul(t,self.w[i])+self.b[i])
            
                
        return t
    def updateTarget(self):
        #print 'TARGET COPY'
        variableList=['layer1','layer2','output']
        self.w=[getVar(self.session,i+'/kernel',False) for i in variableList]
        self.b=[getVar(self.session,i+'/bias',False) for i in variableList]
            
        
    def train(self, episodes_num=EPISODES_NUM):
        global tmp
        # Initialize summary for TensorBoard                         
        summary_writer = tf.summary.FileWriter(self.LOG_DIR)    
        summary = tf.Summary()    
        # Alternatively, you could use animated real-time plots from matplotlib 
        # (https://stackoverflow.com/a/24228275/3284912)
        
        # Initialize the TF session
        self.session = tf.Session()
        self.session.run(tf.global_variables_initializer())
        self.updateTarget()
        self.ReplyMemory=[]
        
        
        playRew=[]
        total_steps=0
        lost=-1
        for episode in range(episodes_num):
          
            state = self.env.reset()

            rew=0
            done = False
            episode_length=0
            while not done:
                
                if np.random.rand()>self.EPSILON:
                    action=self.env.action_space.sample()
                    
                else:
                    action=self.session.run(self.Q,feed_dict={self.x:[state]})
                    action=np.argmax(action)
                    
                #self.EPSILON=1-((1-self.EPSILON)*self.EPSILON_DECAY)
                    
                next_state, reward, done, _ = self.env.step(action)
                rew+=reward
                
                if len(self.ReplyMemory)==self.REPLAY_MEMORY_SIZE:
                    self.ReplyMemory=self.ReplyMemory[1:] #throwing a samples
                self.ReplyMemory.append([state,action,reward,next_state,done*1])
                state=next_state

                if len(self.ReplyMemory)<self.MINIBATCH_SIZE: #get more samples no training until then
                    continue
                samples=random.sample(self.ReplyMemory,self.MINIBATCH_SIZE)
                tmp=samples
                
                states,action_mask,rewards,next_states,dones=getStuff(samples)
                maxqs=np.max(self.targetNetwork(next_states),axis=1)
                
                y_targets=rewards+(1-dones)*self.DISCOUNT_FACTOR*maxqs
                
                _,lost,tp=self.session.run([self.trainer,self.loss,self.Q],feed_dict={self.x:states,
                                                        self.action_mask:action_mask,self.y:y_targets})
                
                
                if total_steps % self.TARGET_UPDATE_FREQ == 0:
                    self.updateTarget()
                total_steps+=1
                episode_length+=1
            playRew.append(rew)
            #print self.EPSILON
            print("Training: Episode = %d, Length = %d, Global step = %d loss =%.4f" % (episode, episode_length, total_steps,lost))
            
            if sum(playRew[-5:])==(200*5): #early stopping
                print 'Early stopping'
                break
        return playRew

    # Simple function to visually 'test' a policy
    def playPolicy(self):
        
        done = False
        steps = 0
        state = self.env.reset()
        
        # we assume the CartPole task to be solved if the pole remains upright for 200 steps
        while not done and steps < 200:     
            self.env.render()                
            q_vals = self.session.run(self.Q, feed_dict={self.x: [state]})
            action = q_vals.argmax()
            state, _, done, _ = self.env.step(action)
            steps += 1
        
        return steps




In [5]:
if __name__ == '__main__':
    tf.reset_default_graph()
    # Create and initialize the model
    dqn = DQN('CartPole-v0')
    dqn.initialize_network()
    
    print("\nStarting training...\n")
    tr=dqn.train()
    print("\nFinished training...\nCheck out some demonstrations\n")
    # Visualize the learned behaviour for a few episodes
    results = []
    for i in range(10):
        episode_length = dqn.playPolicy()
        print("Test steps = ", episode_length)
        results.append(episode_length)
    print("Mean steps = ", sum(results) / len(results))    

    print("\nFinished.")
    print("\nCiao, and hasta la vista...\n")
    

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.

Starting training...

Training: Episode = 0, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 1, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 2, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 3, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 4, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 5, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 6, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 7, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 8, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 9, Length = 0, Global step = 0 loss =-1.0000
Training: Episode = 10, Length = 8, Global step = 8 loss =83.6120
Training: Episode = 11, Length = 12, Global step = 20 loss =65.2091
Training: Episode = 12, Length = 9, Global step = 29 loss =55.4346
Training: Episode = 13, Leng

Training: Episode = 119, Length = 13, Global step = 1452 loss =193.4580
Training: Episode = 120, Length = 17, Global step = 1469 loss =306.2582
Training: Episode = 121, Length = 17, Global step = 1486 loss =109.7306
Training: Episode = 122, Length = 16, Global step = 1502 loss =261.3466
Training: Episode = 123, Length = 17, Global step = 1519 loss =178.4157
Training: Episode = 124, Length = 13, Global step = 1532 loss =88.8155
Training: Episode = 125, Length = 17, Global step = 1549 loss =134.3235
Training: Episode = 126, Length = 13, Global step = 1562 loss =231.5021
Training: Episode = 127, Length = 21, Global step = 1583 loss =125.9019
Training: Episode = 128, Length = 18, Global step = 1601 loss =98.4664
Training: Episode = 129, Length = 36, Global step = 1637 loss =84.6991
Training: Episode = 130, Length = 19, Global step = 1656 loss =96.2522
Training: Episode = 131, Length = 23, Global step = 1679 loss =115.9502
Training: Episode = 132, Length = 31, Global step = 1710 loss =245.2

('Test steps = ', 200)
('Test steps = ', 200)
('Test steps = ', 200)
('Mean steps = ', 200)

Finished.

Ciao, and hasta la vista...

